통계 확률 모델이 있음. 우리가 배운 확률 분포를 기반으로 모수를 추정하는데 그런 확률 모델을 가지고 해왔는데, 거기에 문제를 제대로 해결하기 위해서 통계적 모델이 나옴. 가장 대표적인게 회귀모델. 
머신러닝도 통계적 모델을 기반으로 하지만, 그런 영역을 더 확대해서 통계적 모델로 해결하지 못했던 부분까지 발전함. 통계적인 모델을 넘어서는 머신러닝, 딥러닝 모델들이 있음. 그 이상의 문제들을 해결함.

지금까지 추정의 개념정도만 했고 이제부터는 가설과 검증, 통계 모델을 하고 싸이클런 기반으로 하는 머신러닝 할 예정. (0728 stat1 파일 참고하기)


- 통계적 가설검정은 표본을 사용해서 모집단에 대한 통계적인 판단을 내리는 방법
- 판단의 기준으로 통계적인 표현을 사용하며 간단히 검정이라고 부름
- 평균값의 차이만으로 유의미한 차이를 얻을 수 없으며 샘플사이즈와 데이터의 흩어짐 크기를 가미하는 것이 필요
- 1변량 데이터의 t검정
    - 샘플사이즈가 크고 데이터의 분산이 작고 평균값의 차이가 크다는 세가지 조건을 만족하는 지표를 t값이라고 함
    - t값 = (표본평균 - 모평균) / 표준오차
    - t값이 크면 유의미한 차이가 있다고 판단
- 귀무가설 대립가설
    - 기각 대상이 되는 첫번째 가설을 귀무가설, 귀무가설과 대립되는 가설을 대립가설이라고 함
    - 귀무가설이 기각된다면 즉 귀무가설이 틀렸다고 판단을 하면 유의미한 차이가 있다는 것임
- p값(유의확률): 표본과 귀무가설 간의 모순을 나타내는 지표
    - p값이 작을수록 귀무가설과 표본이 모순된다고 판단
- 유의수준: 귀무가설을 기각하는 기준이 되는 값
    - 유의수준은 위험률이라고 하며 5%를 사용하는 경우가 많음.

In [15]:
### t 검정 : 풀어서

In [5]:
# t 검정 - 귀무가설 : 과자의 무게는 50g이다.
import pandas as pd
junk_food = pd.read_csv('/Users/joy/Documents/GitHub/0Oong/수업내용/stat/5_8_jfw.csv')
jfood = junk_food.weight
jfood.head()

0    58.529820
1    52.353039
2    74.446169
3    52.983263
4    55.876879
Name: weight, dtype: float64

In [7]:
# 표본 평균 구하기
import numpy as np
mu = np.mean(jfood)
mu

55.38496619666667

In [8]:
# 자유도
df = len(jfood)-1
df

19

In [9]:
# 표준 오차 (루트 N분에 편차(시그마))
sigma = np.std(jfood, ddof=1)
se = sigma / np.sqrt(len(jfood))
se

1.9579276805755885

In [11]:
# t value
t_value = (mu-50)/se
t_value

2.7503396831713434

In [13]:
# P value(유의수준)을 통해 t value가 유의수준에 있는지 확인 0.05보다 작으면 귀무가설 기각
from scipy import stats
alpha = stats.t.cdf(t_value, df=df) #얘는 p value 외의 가운데 값
(1-alpha) * 2 #양측검정이기 때문에

0.012725590012524268

### t 검정 : 함수

In [14]:
# 간단하게 하는 t 검정, 50은 모평균
# t 검정 : t값, p값
# p값(유의확률)이 유의수준 0.05보다 작으므로 귀무가설 기각
# '과자의 무게는 50그람이 아니다'라고 할 수 있다. 
stats.ttest_1samp(jfood, 50)

Ttest_1sampResult(statistic=2.750339683171343, pvalue=0.012725590012524182)

- 대응표본 t 검정 : 약을 먹기 전과 후의 체온의 차이를 보는 경우 사용 (동일한 표본집단의 차), 단측검증도 가능(약을 먹은 후 체온이 낮다)
- 독립표본 t 검정 : 큰 바늘로 낚은 물고기의 길이와 작은 바늘로 낚은 물고기의 길이의 차이를 보는 경우 사용. welch 근사법이라고 불리는 방법을 이용하여 p 값 계산(welch 검정)

- 단측검정, 우측검정, 양측검정, 좌측검정 다 가능